In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from itertools import chain
import h2o
import numpy as np
import pandas as pd
h2o.init()
from h2o.estimators.kmeans import H2OKMeansEstimator
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.transforms.decomposition import H2OPCA
pd.options.display.max_columns= None
from h2o.transforms.decomposition import H2OPCA
import seaborn as sns
import matplotlib.pyplot as plt
#from gensim.corpora.Dictionary import load_from_text, doc2bow
#from gensim.corpora import MmCorpus
#from gensim.models.ldamodel import LdaModel

In [ ]:
h2o.remove_all()
#try:
#    h2o.cluster().shutdown()
#except:
#    print("Cluster was already closed.")

In [ ]:
#dont delete
STOP_WORDS = ["i","you","us","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org",
              "like","likes","so","(U)","(U//FOUO)","apos","quot","has","been","do","had","was","were",
             "they","their","them","am","will","than","when","who","where","our","me","your","would",'don','get','more',
             'nga','nro','nsa','new','adf','here','no','see','day','get','why','how','ve','dont','its','ive','work','people',
              'out','which','should','ADF-C','leadership','ADFC']

data=pd.read_excel("2016 ADF-C Climate Survey sample comments.xls",encoding='utf-8')
#data=pd.read_excel("sec_cat.xlsx",sheetname="Sheet1",encoding='utf-8')
h2o_data=h2o.H2OFrame(pd.DataFrame(data['Final Comments']))
h2o_data=h2o_data.ascharacter()
h2o_data

In [ ]:
#Sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score=analyzer.polarity_scores(sentence)
    return(score)

In [ ]:
Dat=data
J=Dat.dropna()['Final Comments'].apply(str).apply(sentiment_analyzer_scores)
J.to_frame("Sentiment").to_csv("sentiment.csv")

In [ ]:
#break into sentenses and clean comments
D=h2o_data.as_data_frame()
D['Final Comments']=D['Final Comments'].apply(str)
#D['Final Comments']
#D=pd.DataFrame(D['Final Comments'].str.split('.', expand=True).values) # I have commented the breaking of sentenses for now
H=D.stack(level=-1)
H=H.to_frame()
H[0]=H[0].str.replace('&;','')
H[0]=H[0].str.replace('apos','')
H[0]=H[0].str.replace(';','')
H[0]=H[0].str.replace('&apos;','')
H[0]=H[0].str.replace('quot','')
H[0]=H[0].str.replace('&','')
H[0]=H[0].str.replace('?','.')
h2o_data=h2o.H2OFrame(H,column_names=['Final Comments'])
h2o_data
h2o_data=h2o_data[h2o_data['Final Comments'] != 'nan']
h2o_data=h2o_data[h2o_data['Final Comments'] != 'None']
h2o_data=h2o_data[h2o_data['Final Comments'] != '']
h2o_data=h2o_data.ascharacter()
#h2o_data

In [ ]:
h2o_data.as_data_frame().to_csv("val.csv")

In [ ]:
#This converts sentenses into vectors
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words
words=tokenize(h2o_data)
print("Build word2vec model")
#gl=h2o.import_file('glove.6B.100d.txt')
gl=h2o.import_file('glove.6B.300d.txt')
w2v_model = H2OWord2vecEstimator(pre_trained=gl)
w2v_model.train(training_frame=words)
survey= w2v_model.transform(words, aggregate_method = "AVERAGE")

In [ ]:
survey

In [ ]:
master=h2o_data.concat(survey)
print(master.nrows)
master

In [ ]:
#Find best K using BIC, this isn't needed but in practice I returned the same number of categories and is statistically more rigerous


Best_BIC=100000
BIC=-999999999999999
K=1
early_stop=int(master.nrows)
while(K<early_stop):
    K=K+1
    kmeans_model=H2OKMeansEstimator(k=K,estimate_k=False,seed=1234)
    m=kmeans_model.train(x=list(range(K)),training_frame=survey)
    BIC=survey.nrows*np.log(kmeans_model.model_performance().totss()/survey.nrows)+K*np.log(survey.nrows)
    print(BIC)
    if Best_BIC>BIC:
        Best_BIC=BIC
    else:
        break



In [ ]:
#we can override K if we wish
#K=9
#kmeans_model=H2OKMeansEstimator(k=K,estimate_k=False,seed=1234)
#m=kmeans_model.train(x=list(range(K)),training_frame=survey)


In [ ]:
kmeans_model

In [ ]:
master=kmeans_model.predict(survey).concat(master)

In [ ]:
master

In [ ]:
f=master.as_data_frame()
f.to_csv("Tagged.csv")

In [ ]:
#look at single word frequencies
Fre=pd.DataFrame()
for i in range(int(master['predict'].max())+1):
    words_nums=h2o.as_list(tokenize(master['Final Comments'][master['predict']==i,:]),use_pandas=True)['C1'].value_counts()
    words_nums=pd.DataFrame(words_nums)
    words_nums.index.name='Category '+str(i)+' Words'
    words_nums.reset_index(inplace=True)
    words_nums.rename(columns={'C1':'Category '+str(i)+' Frequencies'}, inplace=True)
    Fre=pd.concat([Fre,words_nums],axis=1)
Fre.to_csv("Word_frequencies.csv")

In [ ]:
Fre

In [ ]:
#look at bigrams
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

Fre_bigram=pd.DataFrame()
Bigram_nums=pd.DataFrame()

for i in range(int(master['predict'].max())+1):
    Bigram_nums=pd.DataFrame()
    words_nums=h2o.as_list(master['Final Comments'][master['predict']==i,:],use_pandas=True)
    words_nums=pd.DataFrame(words_nums)
    words_nums['bigrams']=words_nums['Final Comments'].map(lambda x: find_ngrams(x.split(" "), 2))
    Bigram_nums['bigrams']=words_nums['bigrams'].to_frame().stack(level=-1)
    Bigram_nums=[w for w in Bigram_nums['bigrams'] if not w in STOP_WORDS]
    Bigram_nums = [item for sublist in Bigram_nums for item in sublist]
    Bigram_nums = [i for i in Bigram_nums if not i[0].lower() in STOP_WORDS and not i[1].lower() in STOP_WORDS ]
    Bigram_nums = [i for i in Bigram_nums if not i[0].lower() in [''] and not i[1].lower() in [''] ]
    Bigram_nums=pd.DataFrame(Bigram_nums,columns=['bigram_1','bigram_2'])
    Bigram_nums['bigrams']=Bigram_nums['bigram_1']+" "+Bigram_nums['bigram_2']
    Bigram_nums['bigrams']=Bigram_nums['bigrams'].str.strip()
    Bigram_nums['bigrams']=Bigram_nums['bigrams'].str.replace('[^\w\s]','')
    Bigram_nums['bigrams']=Bigram_nums['bigrams'].str.replace('\d','')
    Bigram_nums=Bigram_nums["bigrams"].to_frame()
    Bigram_nums=Bigram_nums[Bigram_nums["bigrams"].str.split().apply(len)>1]
    C=Bigram_nums.groupby(['bigrams']).bigrams.agg('count').to_frame()
    C['Text']=C.index
    C['Count']=C['bigrams']
    J=C[['Text','Count']]
    J.reset_index(inplace=True,drop=True)
    J['Count']=J['Count'].apply(int)
    J.sort_values(by=['Count'],ascending=False,inplace=True)
    J.rename(columns={'Count':'Category '+str(i)+' Frequencies'}, inplace=True)
    J.rename(columns={'Text':'Category '+str(i)+' Bigrams'}, inplace=True)
    J.reset_index(inplace=True,drop=True)
    Fre_bigram=pd.concat([Fre_bigram,J],axis=1)
Fre_bigram

The function gloVe ($G$) takes a vector of comments from $\alpha$ and maps them to a cartisian product $R^{n}$. Formally, $G: \alpha^{n} \rightarrow R^{n \times n}$

In [ ]:
words_nums=h2o.as_list(tokenize(h2o_data),use_pandas=True)['C1'].value_counts()
df=pd.DataFrame(words_nums)
df

In [ ]:
#generate graphics

PCA = H2OPCA(k=2,pca_method='Power')

PCA.train(training_frame=master[:,2:102])

master_Decomposed=PCA.predict(master)

In [ ]:
master_Decomposed=master_Decomposed.concat(master[:,0:2])
master_Decomposed
master_Decomposed.as_data_frame().to_csv("viz.csv")

## Generate Graphics

In [ ]:
master_recomposed=master_Decomposed.concat(master)
df_mrc=master_recomposed.as_data_frame()
f, ax= plt.subplots(figsize=(10,10))
sns.set(style="whitegrid")
sns.despine(f, left=True, bottom=True)
sns.scatterplot(x="PC1", y="PC2",data=df_mrc, hue='predict',palette="winter" ,ax=ax, linewidth=0)
plt.title('Comment Similarities')
plt.show()

In [ ]:
df_mrc

In [ ]:
try:
    sns.set(style='darkgrid')
    cat_0=df_mrc.query("predict == 0")
    cat_1=df_mrc.query("predict == 1")
    cat_2=df_mrc.query("predict == 2")
    cat_3=df_mrc.query("predict == 3")
    cat_4=df_mrc.query("predict == 4")
    cat_5=df_mrc.query("predict == 5")
    cat_6=df_mrc.query("predict == 6")
    f,ax = plt.subplots(figsize=(10,10))
    sns.set(style="whitegrid")
    sns.despine(f, left=True, bottom=True)

    ax=sns.kdeplot(cat_0.PC1,cat_0.PC2,cmap="Reds",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_1.PC1,cat_1.PC2,cmap="Blues",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_2.PC1,cat_2.PC2,cmap="rainbow",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_3.PC1,cat_3.PC2,cmap="copper",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_4.PC1,cat_4.PC2,cmap="coolwarm",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_5.PC1,cat_5.PC2,cmap="Dark2",shade=True,shade_lowest=False,legend=True)
    ax=sns.kdeplot(cat_6.PC1,cat_6.PC2,cmap="viridis",shade=True,shade_lowest=False,legend=True)

    red=sns.color_palette("Reds")[-2]
    blue=sns.color_palette("Blues")[-2]
    rainbow=sns.color_palette("rainbow")[-2]
    copper=sns.color_palette("copper")[-2]
    coolwarm=sns.color_palette("coolwarm")[-2]
    Dark2=sns.color_palette("Dark2")[-2]
    viridis=sns.color_palette("viridis")[-2]
except:
    print("Please check number of categories.")




In [ ]:
#Train supervised algorithm for determination as to which words link to which categories

## Test Custom K optimizer- Not for operation- please do not run below cells

In [ ]:
#Find best K using BIC

def find_best_K():
    Best_BIC=100000
    BIC=0
    K=1
    early_stop=int(master.nrows)
    while(K<early_stop):
        K=K+1
        kmeans_model=H2OKMeansEstimator(k=K,estimate_k=False,seed=1234)
        m=kmeans_model.train(x=list(range(K)),training_frame=survey)
        BIC=survey.nrows*np.log(kmeans_model.model_performance().totss()/survey.nrows)+K*np.log(survey.nrows)
        print(BIC)
        if Best_BIC>BIC:
            Best_BIC=BIC
        else:
            break
            
find_best_K()            

In [ ]:
kmeans_model


In [ ]:
#F=between SSE /within SSE, higher is better
F=2865/4655.00
F

In [ ]:
K=int(np.ceil(np.sqrt(master.nrows/2.00)))
kmeans_model=H2OKMeansEstimator(k=K,estimate_k=True,seed=1234)
#kmeans_model=H2OKMeansEstimator(k=16,seed=1234)
m=kmeans_model.train(x=list(range(K)),training_frame=survey)

In [ ]:
kmeans_model

In [ ]:
F=20344/69907.00
F